Imports

In [ ]:
import numpy as np
import pandas as pd

DATA

In [ ]:
train = pd.read_csv('./mnist_train.csv')
test = pd.read_csv('./mnist_test.csv')

In [ ]:
m_train, n_train =  train.shape
m_test, n_test = test.shape
train = np.array(train)
test = np.array(test)

In [ ]:
y_train, x_train = train[:,0], train[:,1:n_train].T/255
y_test, x_test = test[:,0], test[:,1:n_test].T/255